# Modelling

This is the script where I store all my ML model runs

## Library Imports

In [1]:
# Library Imports

#Basics
import pandas as pd
import geopandas as gpd
import numpy as np
import csv

#Shapely / Spatial
from shapely import wkt
import shapely.geometry
from shapely.geometry import Polygon, MultiPolygon

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import OneHotEncoder

#ML from mljar-supervised
from supervised.automl import AutoML

#Warning Supression
import warnings

### Import Datasets

In [31]:
# Read London CSV
all_data_london = pd.read_csv("data/combined_data/lag/all_data_london_lag.csv")

# Read in feature column set
with open("data/combined_data/total_feature_columns_london.csv", 'r') as file:
    lines = file.readlines()

# Reconstruct
feature_columns_london_lag = [''.join(line.strip().split(',')) for line in lines]

# Create non-laged column set
feature_columns_london = [col for col in feature_columns_london_lag if not col.startswith('lag_')]

# ---

# Read Birmingham CSV
all_data_bham = pd.read_csv("data/combined_data/lag/all_data_bham_lag.csv")

# Read in feature column set
with open("data/combined_data/total_feature_columns_bham.csv", 'r') as file:
    lines = file.readlines()

# Reconstruct
feature_columns_bham_lag = [''.join(line.strip().split(',')) for line in lines]

# Create non-lagged column set
feature_columns_bham = [col for col in feature_columns_bham_lag if not col.startswith('lag_')]

# Fix null values ending up in logged variables
all_data_london.fillna(0)
all_data_bham.fillna(0)

,Unnamed: 0,LSOA11CD,LSOA11NM_x,LSOA11NM_y,Unnamed: 2,population,Area,"01 : Crop and animal production, hunting and related service activities",02 : Forestry and logging,03 : Fishing and aquaculture,...,lag_travel,lag_travel_agents,lag_trusts,lag_university_housing,lag_used_vintage_and_consignment,lag_veterinarian,lag_videographer,lag_vitamins_and_supplements,lag_warehouses,lag_window_washing
0,0,E01008881,Birmingham 067A,Birmingham 067A,0.0,1599,lsoa2011:E01008881 : Birmingham 067A,0.0,0.0,0.0,...,0.000000,0.166667,0.0,0.0,0.0,0.0,0.0,0.000000,0.333333,0.0
1,1,E01008882,Birmingham 066A,Birmingham 066A,0.0,1747,lsoa2011:E01008882 : Birmingham 066A,0.0,0.0,0.0,...,0.166667,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.500000,0.0
2,2,E01008883,Birmingham 078A,Birmingham 078A,0.0,1816,lsoa2011:E01008883 : Birmingham 078A,0.0,0.0,0.0,...,0.166667,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
3,3,E01008884,Birmingham 078B,Birmingham 078B,0.0,1870,lsoa2011:E01008884 : Birmingham 078B,0.0,0.0,0.0,...,0.333333,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,0.500000,0.0
4,4,E01008885,Birmingham 076A,Birmingham 076A,0.0,1308,lsoa2011:E01008885 : Birmingham 076A,0.0,0.0,0.0,...,0.000000,0.166667,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,634,E01033646,Birmingham 031I,Birmingham 031I,0.0,1624,lsoa2011:E01033646 : Birmingham 031I,0.0,0.0,0.0,...,0.166667,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,0.000000,0.0
635,635,E01033647,Birmingham 058E,Birmingham 058E,0.0,1398,lsoa2011:E01033647 : Birmingham 058E,0.0,0.0,0.0,...,0.166667,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
636,636,E01033648,Birmingham 084F,Birmingham 084F,0.0,2715,lsoa2011:E01033648 : Birmingham 084F,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
637,637,E01033649,Birmingham 058F,Birmingham 058F,0.0,1801,lsoa2011:E01033649 : Birmingham 058F,0.0,0.0,0.0,...,0.333333,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,0.000000,0.0


In [33]:
### Download POI & auxillary-only feature spaces (No building footprint information)

# London
# Read in POI feature column set
with open("data/combined_data/feature_columns_london_poi.csv", 'r') as file:
    lines = file.readlines()

# Reconstruct
feature_columns_london_poi = [''.join(line.strip().split(',')) for line in lines]

# Birmingham
# Read in POI feature column set
with open("data/combined_data/feature_columns_bham_poi.csv", 'r') as file:
    lines = file.readlines()

# Reconstruct
feature_columns_bham_poi = [''.join(line.strip().split(',')) for line in lines]

### Test Model

In [8]:
# Ignore depracation warnings
warnings.filterwarnings('ignore')

# Create training and testing data
features = feature_columns_london_lag
target = 'log_total_employment'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_london[features], all_data_london[target], test_size=0.2, random_state=3)

automl = AutoML(results_path="ml_results/dummy_models/test", mode='Explain')

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

# fit the model
automl.fit(X_train, y_train)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_log_employment = r2_score(y_test, predictions)
rmse_log_employment = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_log_employment}')
print(f'RMSE: {rmse_log_employment}')

#Save results for plotting
predictions_all = automl.predict(all_data_london[features])
geometries = all_data_london.loc[all_data_london[target].index, 'geometry']

results_test = pd.DataFrame({
    'geometry': geometries,
    'observed': all_data_london[target],
    'predicted': predictions_all,
})

This model has already been fitted. You can use predict methods or select a new 'results_path' for a new 'fit()'.
R^2 Score: 0.7207009026018532
RMSE: 0.678412914276123


## Models

In [12]:
### Employment (log), NO Spatial Lag, London (1)

# Create training and testing data
features = feature_columns_london
target = 'log_total_employment'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_london[features], all_data_london[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/log_employment_london/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_1 = r2_score(y_test, predictions)
rmse_1 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_1}')
print(f'RMSE: {rmse_1}')

#Save results for plotting
predictions_all_1 = automl.predict(all_data_london[features])
london_geometries = all_data_london.loc[all_data_london[target].index, 'geometry']

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': london_geometries,
    'observed': all_data_london[target],
    'predicted': predictions_all_1,
})


AutoML directory: ml_results/log_employment_london/
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['CatBoost', 'Xgboost', 'Random Forest']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'hill_climbing_1', 'hill_climbing_2', 'ensemble']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 3 models
1_Default_Xgboost rmse 0.670873 trained in 41.97 seconds
2_Default_CatBoost rmse 0.665232 trained in 60.41 seconds
3_Default_RandomForest rmse 0.701146 trained in 49.48 seconds
* Step not_so_random will try to check up to 12 models
4_Xgboost rmse 0.67397 trained in 39.45 seconds
8_CatBoost rmse 0.664846 trained in 63.71 seconds
12_RandomForest rmse 0.704978 trained in 80.23 seconds
5_Xgboost rmse 0.686213 trained in 65.79 seconds
9_CatBoost rmse 0.669095 trained in 81.25 seconds
13_RandomForest rmse 0.72372 trained in 53.77 seconds
6_Xgboo

In [13]:
### Employment (log), NO Spatial Lag, Birmingham (2)

# Create training and testing data
features = feature_columns_bham
target = 'log_total_employment'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_bham[features], all_data_bham[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/log_employment_bham/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_2 = r2_score(y_test, predictions)
rmse_2 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_2}')
print(f'RMSE: {rmse_2}')

#Save results for plotting
predictions_all_2 = automl.predict(all_data_bham[features])
bham_geometries = all_data_bham.loc[all_data_bham[target].index, 'geometry']

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': bham_geometries,
    'observed': all_data_bham[target],
    'predicted': predictions_all_2,
})


AutoML directory: ml_results/log_employment_bham/
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['CatBoost', 'Xgboost', 'Random Forest']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'hill_climbing_1', 'hill_climbing_2', 'ensemble']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 3 models
1_Default_Xgboost rmse 0.795926 trained in 6.59 seconds
2_Default_CatBoost rmse 0.80568 trained in 19.24 seconds
3_Default_RandomForest rmse 0.817785 trained in 18.5 seconds
* Step not_so_random will try to check up to 12 models
4_Xgboost rmse 0.816981 trained in 6.21 seconds
8_CatBoost rmse 0.833023 trained in 28.18 seconds
12_RandomForest rmse 0.81406 trained in 29.38 seconds
5_Xgboost rmse 0.819868 trained in 6.84 seconds
9_CatBoost rmse 0.865532 trained in 19.37 seconds
13_RandomForest rmse 0.843877 trained in 25.56 seconds
6_Xgboost rms

NameError: name 'bham_geometries' is not defined

In [19]:
### Employment (log), Spatial Lag, London (3)

# Create training and testing data
features = feature_columns_london_lag
target = 'log_total_employment'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_london[features], all_data_london[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/log_employment_london_lag/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_3 = r2_score(y_test, predictions)
rmse_3 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_3}')
print(f'RMSE: {rmse_3}')

#Save results for plotting
predictions_all_3 = automl.predict(all_data_london[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': london_geometries,
    'observed': all_data_london[target],
    'predicted': predictions_all_3,
})


AutoML directory: ml_results/log_employment_london_lag/
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['CatBoost', 'Xgboost', 'Random Forest']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'hill_climbing_1', 'hill_climbing_2', 'ensemble']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 3 models
1_Default_Xgboost rmse 0.670873 trained in 37.76 seconds
2_Default_CatBoost rmse 0.665232 trained in 58.75 seconds
3_Default_RandomForest rmse 0.701146 trained in 55.51 seconds
* Step not_so_random will try to check up to 12 models
4_Xgboost rmse 0.67397 trained in 48.12 seconds
8_CatBoost rmse 0.664846 trained in 138.48 seconds
12_RandomForest rmse 0.704978 trained in 77.43 seconds
5_Xgboost rmse 0.686213 trained in 52.46 seconds
9_CatBoost rmse 0.669095 trained in 91.01 seconds
13_RandomForest rmse 0.72372 trained in 56.85 seconds
6_

In [20]:
### Employment (log), Spatial Lag, Birmingham (4)

# Create training and testing data
features = feature_columns_bham_lag
target = 'log_total_employment'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_bham[features], all_data_bham[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/log_employment_bham_lag/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_4 = r2_score(y_test, predictions)
rmse_4 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_4}')
print(f'RMSE: {rmse_4}')

#Save results for plotting
predictions_all_4 = automl.predict(all_data_bham[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': bham_geometries,
    'observed': all_data_bham[target],
    'predicted': predictions_all_4,
})


AutoML directory: ml_results/log_employment_bham_lag/
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['CatBoost', 'Xgboost', 'Random Forest']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'hill_climbing_1', 'hill_climbing_2', 'ensemble']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 3 models
1_Default_Xgboost rmse 0.795926 trained in 8.8 seconds
2_Default_CatBoost rmse 0.80568 trained in 19.88 seconds
3_Default_RandomForest rmse 0.817785 trained in 21.62 seconds
* Step not_so_random will try to check up to 12 models
4_Xgboost rmse 0.816981 trained in 9.77 seconds
8_CatBoost rmse 0.833023 trained in 26.56 seconds
12_RandomForest rmse 0.81406 trained in 34.01 seconds
5_Xgboost rmse 0.819868 trained in 7.6 seconds
9_CatBoost rmse 0.865532 trained in 24.66 seconds
13_RandomForest rmse 0.843877 trained in 29.34 seconds
6_Xgboost 

In [ ]:
### Employment Density, NO Spatial Lag, London (5)

# Create training and testing data
features = feature_columns_london
target = 'employment_density'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_london[features], all_data_london[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/employment_density_london/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_5 = r2_score(y_test, predictions)
rmse_5 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_5}')
print(f'RMSE: {rmse_5}')

#Save results for plotting
predictions_all_5 = automl.predict(all_data_london[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': london_geometries,
    'observed': all_data_london[target],
    'predicted': predictions_all_5,
})


In [ ]:
### Employment Density, NO Spatial Lag, Birmingham (6)

# Create training and testing data
features = feature_columns_bham
target = 'employment_density'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_bham[features], all_data_bham[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/employment_density_bham/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_6 = r2_score(y_test, predictions)
rmse_6 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_6}')
print(f'RMSE: {rmse_6}')

#Save results for plotting
predictions_all_6 = automl.predict(all_data_bham[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': bham_geometries,
    'observed': all_data_bham[target],
    'predicted': predictions_all_6,
})


In [ ]:
### Employment Density, Spatial Lag, London (7)

# Create training and testing data
features = feature_columns_london_lag
target = 'employment_density'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_london[features], all_data_london[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/employment_density_london_lag/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_7 = r2_score(y_test, predictions)
rmse_7 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_7}')
print(f'RMSE: {rmse_7}')

#Save results for plotting
predictions_all_5 = automl.predict(all_data_london[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': london_geometries,
    'observed': all_data_london[target],
    'predicted': predictions_all_7,
})


In [ ]:
### Employment Density, Spatial Lag, Birmingham (8)

# Create training and testing data
features = feature_columns_bham_lag
target = 'employment_density'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_bham[features], all_data_bham[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/employment_density_bham_lag/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_8 = r2_score(y_test, predictions)
rmse_8 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_8}')
print(f'RMSE: {rmse_8}')

#Save results for plotting
predictions_all_8 = automl.predict(all_data_bham[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': bham_geometries,
    'observed': all_data_bham[target],
    'predicted': predictions_all_8,
})


In [ ]:
### Office Employment Density , NO Spatial Lag, London (9)

# Create training and testing data
features = feature_columns_london
target = 'office_employment_density'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_london[features], all_data_london[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/office_employment_density_london/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_9 = r2_score(y_test, predictions)
rmse_9 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_9}')
print(f'RMSE: {rmse_9}')

#Save results for plotting
predictions_all_9 = automl.predict(all_data_london[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': london_geometries,
    'observed': all_data_london[target],
    'predicted': predictions_all_9,
})


In [ ]:
### Office Employment Density , NO Spatial Lag, Birmingham (10)

# Create training and testing data
features = feature_columns_bham
target = 'office_employment_density'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_bham[features], all_data_bham[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/office_employment_density_bham/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_10 = r2_score(y_test, predictions)
rmse_10 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_10}')
print(f'RMSE: {rmse_10}')

#Save results for plotting
predictions_all_10 = automl.predict(all_data_bham[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': bham_geometries,
    'observed': all_data_bham[target],
    'predicted': predictions_all_10,
})


In [ ]:
### Office Employment Density, Spatial Lag, London (11)

# Create training and testing data
features = feature_columns_london_lag
target = 'office_employment_density'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_london[features], all_data_london[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/office_employment_density_london_lag/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_11 = r2_score(y_test, predictions)
rmse_11 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_11}')
print(f'RMSE: {rmse_11}')

#Save results for plotting
predictions_all_11 = automl.predict(all_data_london[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': london_geometries,
    'observed': all_data_london[target],
    'predicted': predictions_all_11,
})


In [ ]:
### Office Employment Density (log), Spatial Lag, Birmingham (12)

# Create training and testing data
features = feature_columns_bham_lag
target = 'office_employment_density'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_bham[features], all_data_bham[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/office_employment_density_bham_lag/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_12 = r2_score(y_test, predictions)
rmse_12 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_12}')
print(f'RMSE: {rmse_12}')

#Save results for plotting
predictions_all_12 = automl.predict(all_data_bham[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': bham_geometries,
    'observed': all_data_bham[target],
    'predicted': predictions_all_12,
})


In [ ]:
### Employment (log), NO Spatial Lag, London, POI only (13)

# Create training and testing data
features = feature_columns_london_poi
target = 'log_total_employment'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_london[features], all_data_london[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/log_employment_london_poi/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_13 = r2_score(y_test, predictions)
rmse_13 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_13}')
print(f'RMSE: {rmse_13}')

#Save results for plotting
predictions_all_13 = automl.predict(all_data_london[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': london_geometries,
    'observed': all_data_london[target],
    'predicted': predictions_all_13,
})


In [ ]:
### Employment (log), NO Spatial Lag, Brimingham, POI only (14)

# Create training and testing data
features = feature_columns_bham_poi
target = 'log_total_employment'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_bham[features], all_data_bham[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/log_employment_bham_poi/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_14 = r2_score(y_test, predictions)
rmse_14 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_14}')
print(f'RMSE: {rmse_14}')

#Save results for plotting
predictions_all_14 = automl.predict(all_data_bham[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': bham_geometries,
    'observed': all_data_bham[target],
    'predicted': predictions_all_14,
})


In [ ]:
### Employment Density, NO Spatial Lag, London, POI only (15)


# Create training and testing data
features = feature_columns_london_poi
target = 'employment_density'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_london[features], all_data_london[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/employment_density_london_poi/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_15 = r2_score(y_test, predictions)
rmse_15 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_15}')
print(f'RMSE: {rmse_15}')

#Save results for plotting
predictions_all_15 = automl.predict(all_data_london[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': london_geometries,
    'observed': all_data_london[target],
    'predicted': predictions_all_15,
})


In [ ]:
### Employment Density, NO Spatial Lag, Brimingham, POI only (16)

# Create training and testing data
features = feature_columns_bham_poi
target = 'employment_density'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_bham[features], all_data_bham[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/employment_density_bham_poi/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_16 = r2_score(y_test, predictions)
rmse_16 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_16}')
print(f'RMSE: {rmse_16}')

#Save results for plotting
predictions_all_16 = automl.predict(all_data_bham[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': bham_geometries,
    'observed': all_data_bham[target],
    'predicted': predictions_all_16,
})


In [ ]:
### Office Employment Density, NO Spatial Lag, London, POI only (17)


# Create training and testing data
features = feature_columns_london_poi
target = 'office_employment_density'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_london[features], all_data_london[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/office_employment_density_london_poi/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_17 = r2_score(y_test, predictions)
rmse_17 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_17}')
print(f'RMSE: {rmse_17}')

#Save results for plotting
predictions_all_17 = automl.predict(all_data_london[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': london_geometries,
    'observed': all_data_london[target],
    'predicted': predictions_all_17,
})


In [ ]:
### Office Employment Density, NO Spatial Lag, Birmingham, POI only (18)

# Create training and testing data
features = feature_columns_bham_poi
target = 'office_employment_density'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data_bham[features], all_data_bham[target], test_size=0.2, random_state=3)

automl = AutoML(
    results_path ='ml_results/office_employment_density_bham_poi/',
    algorithms=["CatBoost", "Xgboost", "Random Forest"],
    model_time_limit=1*60,
    start_random_models=5,
    hill_climbing_steps=2,
    top_models_to_improve=3,
    features_selection=False,
    stack_models=False,
    train_ensemble=True,
    explain_level=1,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)

# fit the model
automl.fit(X_train, y_train)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

predictions = automl.predict(X_test)
predictions = predictions.astype(np.float32)

# Fix NaN values
y_test = y_test.fillna(0)

r2_18 = r2_score(y_test, predictions)
rmse_18 = np.sqrt(mean_squared_error(y_test, predictions))

print(f'R^2 Score: {r2_18}')
print(f'RMSE: {rmse_18}')

#Save results for plotting
predictions_all_10 = automl.predict(all_data_bham[features])

results_office_density_cleaned_perform = pd.DataFrame({
    'geometry': bham_geometries,
    'observed': all_data_bham[target],
    'predicted': predictions_all_18,
})
